# Kuramoto example: Continuous vs Discrete Energy Landscape

This notebook simulates BOLD-like signals from a Kuramoto + hemodynamic
model, then compares discrete vs continuous energy landscape approaches
for recovering hidden regimes.


In [1]:
import os, sys

sys.path.append(os.path.abspath(".."))


import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

from continuous_energy_landscape import ContinuousEnergyLandscape
from simulations.kuramoto_sim_simple import simulate_kuramoto_example
from metrics_energy_landscape import compute_metrics_all
from discrete_energy import binarize, ener_calculate_pca_bin



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/tranminhtriet/opt/anaconda3/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/tranminhtriet/opt/anaconda3/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/Users/tranminhtriet/opt/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/tranminhtriet/opt/anaconda3/lib/python3.9/site-packages/traitlets/config/application.py", line 992, i

## 1. Simulate Kuramoto-BOLD data


In [2]:
K = 3
d = 12    
T = 3000
seed = 0

sim, X, g_true = simulate_kuramoto_example(
    K=K,
    d=d,
    T=T,
    dwell=160,   # slightly longer dwell
    sigma=0.6,   # keep some noise
    seed=seed,
)

P_true = sim.P
print("X shape:", X.shape)
print("g_true shape:", g_true.shape)



X shape: (3000, 12)
g_true shape: (3000,)


## 2. Discrete energy landscape (DEL) baseline

As in the SLDS example, we fit an Ising model on binarized data and
            cluster the resulting energy time series.


In [3]:
X_bin = binarize(X.T)
h_del, W_del, E_del = ener_calculate_pca_bin(X_bin)
print("Discrete energy shape:", E_del.shape)

E_feat = E_del.reshape(-1, 1)
km_del = KMeans(n_clusters=K, random_state=seed)
g_del = km_del.fit_predict(E_feat)
print("DEL labels shape:", g_del.shape)


[MEM] it=  200 max|mom-res|=3.361e-01, step=0.2
[MEM] it=  400 max|mom-res|=3.369e-01, step=0.197
[MEM] it=  600 max|mom-res|=3.326e-01, step=0.194
[MEM] it=  800 max|mom-res|=3.264e-01, step=0.191
[MEM] it= 1000 max|mom-res|=3.209e-01, step=0.188
[MEM] it= 1200 max|mom-res|=3.153e-01, step=0.185
[MEM] it= 1400 max|mom-res|=3.085e-01, step=0.183
[MEM] it= 1600 max|mom-res|=2.902e-01, step=0.18
[MEM] it= 1800 max|mom-res|=2.710e-01, step=0.177
[MEM] it= 2000 max|mom-res|=2.339e-01, step=0.175
[MEM] it= 2200 max|mom-res|=1.880e-01, step=0.172
[MEM] it= 2400 max|mom-res|=1.242e-01, step=0.169
[MEM] it= 2600 max|mom-res|=1.376e-02, step=0.167
[MEM] it= 2800 max|mom-res|=8.297e-04, step=0.164
[MEM] it= 3000 max|mom-res|=7.703e-04, step=0.162
[MEM] it= 3200 max|mom-res|=7.159e-04, step=0.159
[MEM] it= 3400 max|mom-res|=6.673e-04, step=0.157
[MEM] it= 3600 max|mom-res|=6.240e-04, step=0.155
[MEM] it= 3800 max|mom-res|=5.855e-04, step=0.152
[MEM] it= 4000 max|mom-res|=5.511e-04, step=0.15
[MEM

/Users/tranminhtriet/opt/anaconda3/lib/python3.9/site-packages/threadpoolctl.py:1223: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


## 3. Continuous energy landscape (CEL) model


In [4]:
cel = ContinuousEnergyLandscape(
    hidden_channels=64,
    rank=32,
    delta=0.10,
    eps=1e-2,
    lambda_reg=1e-2,
    lr=1e-3,
    weight_decay=0.0,
    max_epochs=300,
    clip_grad=1.0,
    verbose=True,
    device="cpu",
    seed=seed,
)

out = cel.fit(X)
E_cel = cel.predict_energy(X)
print("CEL energy shape:", E_cel.shape)

S = cel.S_
eigvals, eigvecs = np.linalg.eigh(S)
v_top = eigvecs[:, np.argmax(eigvals)]
z_cel = (X @ v_top).reshape(-1, 1)

F_cel = np.hstack([E_cel.reshape(-1, 1), z_cel])
km_cel = KMeans(n_clusters=K, random_state=seed)
g_cel = km_cel.fit_predict(F_cel)
print("CEL labels shape:", g_cel.shape)


[epoch 0000] loss=338991.343750
[epoch 0100] loss=16507.648438
[epoch 0200] loss=5400.143066
[epoch 0299] loss=2717.717773
CEL energy shape: (3000,)
CEL labels shape: (3000,)


## 4. Metrics: BR, TMA, SDA


In [5]:
metrics_del = compute_metrics_all(g_true, g_del, K=K, P_true=P_true, X=X)
metrics_cel = compute_metrics_all(g_true, g_cel, K=K, P_true=P_true, X=X)

print("DEL metrics:", metrics_del)
print("CEL metrics:", metrics_cel)

print("DEL ARI:", adjusted_rand_score(g_true, g_del))
print("CEL ARI:", adjusted_rand_score(g_true, g_cel))


DEL metrics: {'BR_ARI': 0.0362001226688772, 'TMA': 0.8017874277755512, 'SDA': 0.879747818376874}
CEL metrics: {'BR_ARI': 0.09561992358962867, 'TMA': 0.8952182135625999, 'SDA': 0.9375414473966706}
DEL ARI: 0.0362001226688772
CEL ARI: 0.09561992358962867
